In [1]:
STREAM_NAME = "TEST"
CONSUMER = "app-01"

from jsm.api.client import Client as JS

js = JS()
await js.connect()

- List consumers

In [2]:
await js.consumer_list(STREAM_NAME)

IoNatsJetstreamApiV1ConsumerListResponse(type='io.nats.jetstream.api.v1.consumer_list_response', total=0, offset=0, limit=256, consumers=[])

- Create a consumer

In [3]:
await js.consumer_durable_create(STREAM_NAME, CONSUMER)

IoNatsJetstreamApiV1ConsumerCreateResponse(type='io.nats.jetstream.api.v1.consumer_create_response', stream_name='TEST', name='app-01', config=Config(durable_name='app-01', deliver_subject=None, deliver_policy='last', ack_policy=<AckPolicy.explicit: 'explicit'>, ack_wait=30000000000, max_deliver=-1, filter_subject=None, replay_policy=<ReplayPolicy.instant: 'instant'>, sample_freq=None, rate_limit_bps=None, max_ack_pending=20000, idle_heartbeat=None, flow_control=None, max_waiting=512), created='2021-07-25T23:18:47.9586752Z', delivered=Delivered(consumer_seq=0, stream_seq=0), ack_floor=AckFloor(consumer_seq=0, stream_seq=0), num_ack_pending=0, num_redelivered=0, num_waiting=0, num_pending=0, cluster=Cluster(name=None, leader='NBTGNZD4UJQXTO6LGPYL7M6RBXOLDF3HY6MKV5IIP4AOBU6EQVN2SUDO', replicas=None))

- Get consumer infos

In [4]:
response = await js.consumer_info(STREAM_NAME, CONSUMER)

In [5]:
response.dict()

{'type': 'io.nats.jetstream.api.v1.consumer_info_response',
 'stream_name': 'TEST',
 'name': 'app-01',
 'config': {'durable_name': 'app-01',
  'deliver_subject': None,
  'deliver_policy': 'last',
  'ack_policy': <AckPolicy.explicit: 'explicit'>,
  'ack_wait': 30000000000,
  'max_deliver': -1,
  'filter_subject': None,
  'replay_policy': <ReplayPolicy.instant: 'instant'>,
  'sample_freq': None,
  'rate_limit_bps': None,
  'max_ack_pending': 20000,
  'idle_heartbeat': None,
  'flow_control': None,
  'max_waiting': 512},
 'created': '2021-07-25T23:18:47.9586752Z',
 'delivered': {'consumer_seq': 0, 'stream_seq': 0},
 'ack_floor': {'consumer_seq': 0, 'stream_seq': 0},
 'num_ack_pending': 0,
 'num_redelivered': 0,
 'num_waiting': 0,
 'num_pending': 0,
 'cluster': {'name': None,
  'leader': 'NBTGNZD4UJQXTO6LGPYL7M6RBXOLDF3HY6MKV5IIP4AOBU6EQVN2SUDO',
  'replicas': None}}

- Publish a message into a subject watched by the consumer

In [6]:
await js.publish("test.demo", b"1")

- Fetch the message

**WARNING**: `auto_hack` is set to False by default, as in the example. It means that messages must be acknowledged manually.

In [7]:
last_msg = await js.consumer_pull_next(STREAM_NAME, CONSUMER, auto_ack=False)
last_msg.data

b'1'

- Acknowledge the message

In [8]:
await js.acknowledge(last_msg)

If you don't want to manually acknowledge messages, set `auto_ack` to `True`.

- Iterate over messages. While async iterator is running, messages are sent using `nats` command line tool. By default iterator will wait for messages forever, this can be changed using `max_msgs` keyword argument. If it is set, once the maximum number of message has been received, subscription is stopped.

In [9]:
async for message in js.consumer_pull_msgs(STREAM_NAME, CONSUMER, max_msgs=3):
    # Do some work with the message
    print(message)
    # Ackownledge the message.
    # If message is not acknowledged, it will eventually be resent
    await js.acknowledge(message)

<Msg: subject='test.demo' reply='$JS.ACK.TEST.app-01.1.2.2.1627255143438309500.0' data='2...'>
<Msg: subject='test.demo' reply='$JS.ACK.TEST.app-01.1.3.3.1627255144618338100.0' data='3...'>
<Msg: subject='test.demo' reply='$JS.ACK.TEST.app-01.1.4.4.1627255145645437800.0' data='4...'>


- A fourth message was sent so it's still pending

In [10]:
await js.consumer_info(STREAM_NAME, CONSUMER)

IoNatsJetstreamApiV1ConsumerInfoResponse(type='io.nats.jetstream.api.v1.consumer_info_response', stream_name='TEST', name='app-01', config=Config(durable_name='app-01', deliver_subject=None, deliver_policy='last', ack_policy=<AckPolicy.explicit: 'explicit'>, ack_wait=30000000000, max_deliver=-1, filter_subject=None, replay_policy=<ReplayPolicy.instant: 'instant'>, sample_freq=None, rate_limit_bps=None, max_ack_pending=20000, idle_heartbeat=None, flow_control=None, max_waiting=512), created='2021-07-25T23:18:47.9586752Z', delivered=Delivered(consumer_seq=4, stream_seq=4), ack_floor=AckFloor(consumer_seq=4, stream_seq=4), num_ack_pending=0, num_redelivered=0, num_waiting=0, num_pending=0, cluster=Cluster(name=None, leader='NBTGNZD4UJQXTO6LGPYL7M6RBXOLDF3HY6MKV5IIP4AOBU6EQVN2SUDO', replicas=None))

- We can pull it easily

In [11]:
await js.consumer_pull_next(STREAM_NAME, CONSUMER)

<Msg: subject='test.demo' reply='$JS.ACK.TEST.app-01.1.5.5.1627255146774986100.0' data='5...'>